# Knowledge and Data: Practical Assignment 3 
## RDF Data, RDFS knowledge and inferencing 

YOUR NAME: Sebastião Manuel Inácio Rosalino

YOUR VUNetID: sxx209

*(If you do not provide your name and VUNetID we will not accept your submission).* 

### Learning objectives

At the end of this exercise you should be able to:

1. Access local an external data via SPARQL both from within a python programming environment and stand-alone with a GUI, such as [YASGUI](https://yasgui.triply.cc/), and this way integrate data from different sources  
2. Model your own first knowledge base, in this case an RDF Schema knowledge graph
3. Implement inference rules 

Follow this Notebook step-by-step. 

Of course, you can do the exercises in any Programming Editor of your liking. 
But you do not have to. Feel free to simply write code in the Notebook. When 
everythink is filled in and works, safe the Notebook and submit it 
as a Jupyter Notebook, i.e. with an ipynb extension. Please use as name of the 
Notebook your studentID+Assignment3.ipynb.  

Other than in courses dedicated to programming we will not evaluate the style
of the programs. But we will test your programs on other data than we provide, 
and your program should give the correct answers to those test-data as well. 

Before you start, you need to:

- **Install the *rdflib* Python package:** *pip install rdflib* (should already be installed from the previous assignment)
- **Install the *SPARQLWrapper* Python package:** *pip install SPARQLWrapper*
- **Install the free edition of the GraphDB Triplestore:** please follow this short [GraphDB tutorial](https://github.com/ucds-vu/knowledge-data-vu/blob/master/Tutorials/Preliminaries/tutorial-GraphDB.md). 

Then, add the file example-from-slides.ttl to a newly created database, say called assignment-3. 

**Note that you should have an active internet connection to run the code in this notebook.**

In [19]:
# install library
%pip install SPARQLWrapper

Note: you may need to restart the kernel to use updated packages.


## Task 1: (3 points) Integrate Local and External Data

You can integrate SPARQL queries into your Python code by using the *RDFLib* and *SPARQLWrapper* libraries. 

The following code accesses the DBPedia knowledge graph using its SPARQL endpoint, and returns the result of the SPARQL query requesting all the labels asserted to Amsterdam (test it!)  

In [20]:
# This code only works if you are online

from rdflib import Graph, RDF, RDFS, Namespace, Literal, URIRef
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?cityName
    WHERE { 
        <http://dbpedia.org/resource/Amsterdam> rdfs:label ?cityName 
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print(result["cityName"]["value"])  

Amsterdam
أمستردام
حكومة أمستردام
Amsterdam
Amsterdam
Άμστερνταμ (δήμος)
Άμστερνταμ
Amsterdam
Amsterdamo
Ámsterdam
Amsterdam
Amsterdam (commune)
Amsterdam
Amstardam
Amsterdam
アムステルダム
Amsterdam
암스테르담
Amsterdam (gemeente)
Amsterdam
Amsterdam
Amesterdão
Амстердам
Amsterdam
Амстердам
阿姆斯特丹


Your task is now the following:

A: Write a SPARQL query that extracts all the cities from your local knowledge graph (constructed by loading the file example-from-slides.ttl) 

B: Find the number of inhabitants of these cities and the longitude and latitude information (if available) from DBPedia.

C: Merge the triples from example-from-slides.ttl with the information extracted from DBpedia + Save all these triples into a new file 'extended-example.ttl' + Print all triples in Turtle Syntax.

For your convenience, we already wrote the following functions that might be useful to complete this task. 
In addition, we have loaded and printed the 'example-from-slides.ttl' dataset.

In [21]:
from rdflib import Graph, RDF, Namespace, Literal, URIRef
from SPARQLWrapper import SPARQLWrapper, JSON


# Loads the data from a certain file given as input in Turtle syntax into the Graph g  
# -------------------------
def load_graph(graph, filename):
    with open(filename, 'r') as f:
        graph.parse(f, format='turtle')
        

# Prints a certain graph given as input in Turtle syntax
# -------------------------
def serialize_graph(myGraph):
     print(myGraph.serialize(format='turtle'))
        

# Saves the Graph g in Turtle syntax to a certain file given as input
# -------------------------
def save_graph(myGraph, filename):
    with open(filename, 'w') as f:
        myGraph.serialize(filename, format='turtle')
        
    
# Changes the namespace of a certain URI given as input to a DBpedia URI 
# Example: transformToDBR("http://example.com/kad2020/Amsterdam") returns "http://dbpedia.org/resource/Amsterdam"
# -------------------------
def transformToDBR(uri):
    if isinstance(uri, Literal):
        # changes the literal to uppercase so that the object with the same name refers to an object and not the string
        return uri.upper()
    components = g.namespace_manager.compute_qname(uri)
    return "http://dbpedia.org/resource/%s"%(components[2])

# -------------------------

g = Graph()
load_graph(g, 'example-from-slides.ttl')
serialize_graph(g)


# Don't forget to run this cell before continuing the task.


@prefix ex: <http://example.com/kad/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:Netherlands a ex:Country ;
    ex:contains ex:Ijsselmeer ;
    ex:containsCity ex:Rotterdam ;
    ex:hasCapital ex:Amsterdam ;
    ex:hasName "The Netherlands" ;
    ex:neighbours ex:Belgium .

ex:hasCapital rdfs:range ex:Capital ;
    rdfs:subPropertyOf ex:containsCity .

ex:neighbours rdfs:subPropertyOf ex:closeBy .

ex:Amsterdam a ex:Capital ;
    ex:closeBy ex:Germany .

ex:Belgium a ex:Country .

ex:EuropeanCountry rdfs:subClassOf ex:Country .

ex:Germany a ex:EuropeanCountry ;
    ex:hasCapital ex:Berlin .

ex:closeBy rdfs:domain ex:Location ;
    rdfs:range ex:Location .

ex:containsCity rdfs:domain ex:Country ;
    rdfs:range ex:City ;
    rdfs:subPropertyOf ex:contains .

ex:Capital rdfs:subClassOf ex:City .

ex:City rdfs:subClassOf ex:Location .

ex:Country rdfs:subClassOf ex:Location .




### A: Write a SPARQL query that finds all the cities in the dataset

As you cannot directly use class City, you will have to find those cities in the dataset (example-from-slides.ttl) using implicit information that can be deduced from the domain and ranges of the relations (e.g. things in a hasCapital relation are capitals and a capital is a city, etc.).

Save all the cities returned from the SPARQL query into the empty set "cities". 

In [22]:
cities = set()

all_cities = g.query("""

    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
    PREFIX dbo: <http://dbpedia.org/ontology/> 
    PREFIX dbr: <http://dbpedia.org/resource/> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX ex: <http://example.com/kad/> 
    
    SELECT DISTINCT ?city
    WHERE { 
        {?city rdf:type ex:Capital .}
    UNION {?country ex:hasCapital ?city .}
    UNION {?country ex:containsCity ?city .} }
    
""")

for city_query in all_cities:
    cities.add(city_query[0])
    
for city in cities:
    print(city)

http://example.com/kad/Berlin
http://example.com/kad/Amsterdam
http://example.com/kad/Rotterdam


### B: For each city, find from DBpedia its longitude & latitude, and its number of inhabitants (if available)

Don't forget to adapt the namespace of the cities in your dataset when querying DBpedia, using the above function *transformToDBR(uri)*. Also note that namespaces should never use the *https* protocol.

The empty graph h should only contain the triples extracted from DBpedia, but added to the URIs with the 'ex' namespace. 
An example of a triple in h is the following triple: 
       
       ex:Amsterdam dbo:populationTotal "872680"^^xsd:nonNegativeInteger .

In [23]:
h = Graph()

def query_cities(city):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(f"""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>

        SELECT ?latitude ?longitude ?population
        WHERE {{ 
            OPTIONAL {{ <{city}> dbo:populationTotal ?population .}}
            OPTIONAL {{ <{city}> geo:lat ?latitude .}}
            OPTIONAL {{ <{city}> geo:long ?longitude .}}
        }}
    """)
    
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results["results"]["bindings"][0]


for city in cities:
    cleaned_city = transformToDBR(city)   
    final_dict = query_cities(cleaned_city)
    
    if 'latitude' in final_dict.keys():
        h.add((city, URIRef("https://dbpedia.org/property/geo/lat"), Literal(final_dict["latitude"]["value"], datatype=final_dict["latitude"]["datatype"]))) 

    if 'longitude' in final_dict.keys():
        h.add((city, URIRef("https://dbpedia.org/property/geo/long"), Literal(final_dict["longitude"]["value"], datatype=final_dict["longitude"]["datatype"]))) 

    if 'population' in final_dict.keys():
        h.add((city, URIRef("https://dbpedia.org/ontology/populationTotal"), Literal(final_dict["population"]["value"], datatype=final_dict["population"]["datatype"])))
                          

serialize_graph(h)

@prefix ns1: <https://dbpedia.org/property/geo/> .
@prefix ns2: <https://dbpedia.org/ontology/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.com/kad/Amsterdam> ns2:populationTotal "872680"^^xsd:nonNegativeInteger ;
    ns1:lat "52.3667"^^xsd:float ;
    ns1:long "4.9"^^xsd:float .

<http://example.com/kad/Berlin> ns2:populationTotal "3769495"^^xsd:nonNegativeInteger ;
    ns1:lat "52.52"^^xsd:float ;
    ns1:long "13.405"^^xsd:float .

<http://example.com/kad/Rotterdam> ns2:populationTotal "651157"^^xsd:nonNegativeInteger ;
    ns1:lat "51.9167"^^xsd:float ;
    ns1:long "4.5"^^xsd:float .




### C: Save your results

- Merge the triples from example-from-slides.ttl with the information extracted from DBpedia.
- Save all these triples into a new file 'extended-example.ttl'. It is not necessary to submit this file.
- Print all triples in Turtle Syntax.


In [24]:
# Merging process of h into the file 'example-from-slides.ttl'

load_graph(h, 'example-from-slides.ttl')

# Saving the merged graph into the new file 'extended-example.ttl'

save_graph(h, 'extended-example.ttl')

# Printing all triples in Turtle Syntax

serialize_graph(h)

@prefix ex: <http://example.com/kad/> .
@prefix ns1: <https://dbpedia.org/property/geo/> .
@prefix ns2: <https://dbpedia.org/ontology/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Netherlands a ex:Country ;
    ex:contains ex:Ijsselmeer ;
    ex:containsCity ex:Rotterdam ;
    ex:hasCapital ex:Amsterdam ;
    ex:hasName "The Netherlands" ;
    ex:neighbours ex:Belgium .

ex:hasCapital rdfs:range ex:Capital ;
    rdfs:subPropertyOf ex:containsCity .

ex:neighbours rdfs:subPropertyOf ex:closeBy .

ex:Amsterdam a ex:Capital ;
    ex:closeBy ex:Germany ;
    ns2:populationTotal "872680"^^xsd:nonNegativeInteger ;
    ns1:lat "52.3667"^^xsd:float ;
    ns1:long "4.9"^^xsd:float .

ex:Belgium a ex:Country .

ex:Berlin ns2:populationTotal "3769495"^^xsd:nonNegativeInteger ;
    ns1:lat "52.52"^^xsd:float ;
    ns1:long "13.405"^^xsd:float .

ex:EuropeanCountry rdfs:subClassOf ex:Country .

ex:Germany a ex:EuropeanCountry ;
  

## Task 2: (3 points)  Implement Basic Inferencing Rules 

In the lecture we showed that the RDFS inference rules can be used to infer new knowledge. For example, infer class membership based on _rdfs:domain_ or infer relationships between subjects and objects based on _rdfs:subPropertyOf_. 

Create rules to inference class membership based on the RDF Schema language features 
*	For example: infer that an instance belongs to a class because of domain and range restrictions
*	For example: infer that an instance belongs to a (super)class because it also belongs to a subclass

We implemented the __rdfs2__ rule. You should implement the 5 following remaining rules:  

*     (rdfs2) If G contains the triples (aaa rdfs:domain xxx.) and (uuu aaa yyy.)  then infer the triple (uuu rdf:type xxx.)
*     (rdfs3) If G contains the triples (aaa rdfs:range xxx.) and (uuu aaa vvv.) then infer the triple (vvv rdf:type xxx .)
*     (rdfs5) If G contains the triples (uuu rdfs:subPropertyOf vvv.) and (vvv rdfs:subPropertyOf xxx.) then infer the triple
(uuu rdfs:subPropertyOf xxx.) 
*     (rdfs7) If G contains the triples (aaa rdfs:subPropertyOf bbb.) and (uuu aaa yyy.) then infer the triple (uuu bbb yyy) 
*     (rdfs9) If G contains the triples (uuu rdfs:subClassOf xxx.) and (vvv rdf:type uuu.) then infer the triple
 (vvv rdf:type xxx.)   -> this one was not mentioned in the lecture, but is a very important one. 
*     (rdfs11) If G contains the triples (uuu rdfs:subClassOf vvv.) and (vvv rdfs:subClassOf xxx.) then infer the triple
(uuu rdfs:subClassOf xxx.)


Run your rule reasoner on your knowledge graph.

In [25]:
def myRDFSreasoner(myGraph):
    inferredTriples = 0
    for sbj, prd, obj in myGraph:

        # --- rdfs2 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#domain"))):
            generator = myGraph.subject_objects(URIRef(sbj))
            for s, o in generator:
                inferredTriples += 1
                print("(rdfs 2) ", s, "rdf:type", obj)
        
        
        # --- rdfs3 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#range"))):
            generator = myGraph.subject_objects(URIRef(sbj))
            for s, o in generator:
                inferredTriples += 1
                print("(rdfs 3) ", o, "rdf:type", obj)
        
        
        # --- rdfs5 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subPropertyOf"))):
            generator = myGraph.triples((URIRef(obj), URIRef("http://www.w3.org/2000/01/rdf-schema#subPropertyOf"), None))
            for s, p, o in generator:
                inferredTriples += 1
                print("(rdfs 5) ", sbj, "rdfs:subPropertyOf", o)

                
        # --- rdfs7 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subPropertyOf"))):
            generator = myGraph.triples((None, URIRef(sbj), None))
            for s, p, o in generator:
                inferredTriples += 1
                print("(rdfs 7) ", s, obj, o)  
        
        
        # --- rdfs9 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf"))):
            generator = myGraph.triples((None, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), URIRef(sbj)))
            for s, p, o in generator:
                inferredTriples += 1
                print("(rdfs 9) ", s, "rdf:type", obj) 
        
        
        # --- rdfs11 ---
        if (prd.eq(URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf"))):
            generator = myGraph.triples((URIRef(obj), URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf"), None))
            for s, p, o in generator:
                inferredTriples += 1
                print("(rdfs 11) ", sbj, "rdfs:subClassOf", o) 
        
        
    print("---------------------------------")
    print("Number of inferred triples:", inferredTriples)
    print("---------------------------------")
    
myRDFSreasoner(g)  # test your reasoner

(rdfs 9)  http://example.com/kad/Germany rdf:type http://example.com/kad/Country
(rdfs 11)  http://example.com/kad/EuropeanCountry rdfs:subClassOf http://example.com/kad/Location
(rdfs 7)  http://example.com/kad/Netherlands http://example.com/kad/closeBy http://example.com/kad/Belgium
(rdfs 5)  http://example.com/kad/hasCapital rdfs:subPropertyOf http://example.com/kad/contains
(rdfs 7)  http://example.com/kad/Netherlands http://example.com/kad/containsCity http://example.com/kad/Amsterdam
(rdfs 7)  http://example.com/kad/Germany http://example.com/kad/containsCity http://example.com/kad/Berlin
(rdfs 2)  http://example.com/kad/Amsterdam rdf:type http://example.com/kad/Location
(rdfs 7)  http://example.com/kad/Netherlands http://example.com/kad/contains http://example.com/kad/Rotterdam
(rdfs 3)  http://example.com/kad/Germany rdf:type http://example.com/kad/Location
(rdfs 3)  http://example.com/kad/Rotterdam rdf:type http://example.com/kad/City
(rdfs 3)  http://example.com/kad/Amsterdam

## Task 3: (2 points) Build your very own RDFS knowledge graph. 


Define a small RDF Schema vocabulary in Turtle. You can choose your own domain (e.g. movies, geography, sports) respecting all the following rules:
*	The schema should define at least 4 classes, 4 properties, and 4 instances.
*   The properties should be used to relate the instances (i.e., object-type relations)
*	The instances should be members of at least one of the 4 defined classes
*	All resources should have an rdfs:label attribute in a suitable language.

You should use (at least) the following language features of RDF and RDFS:
* 	rdf:type (or 'a')
* 	rdfs:subClassOf
* 	rdfs:subPropertyOf
* 	rdfs:domain and rdfs:range
*	rdfs:label

Be sure to define the 'rdf:' and 'rdfs:' namespace prefixes for RDF and RDF Schema in your file (perhaps have a look at http://prefix.cc)

For creating your vocabulary, you can either use a text editor, or add the axioms directly (programatically) to your Knowledge Graph as you did last week. 

Play around with the inference rules you have created in the previous task to make sure that you added some implicit knowledge, that becomes "visible" via inferencing (this will be useful for the next task). 

Finally:
- Add the knowledge you created into the RDFlib graph datastructure *myRDFSgraph*, 
- Print *myRDFSgraph* in Turtle so that we can check your "design"
- Save *myRDFSgraph* into a new file 'myRDFSgraph.ttl' (it is not necessary to submit this file)

In [26]:
myRDFSgraph = Graph()


# Nomination of the prefixes

ex = Namespace("http://example.com/kad/")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")


# Classes

myRDFSgraph.add((ex.Town, rdf.type, ex.Class))
myRDFSgraph.add((ex.District, rdf.type, ex.Class))
myRDFSgraph.add((ex.Region, rdf.type, ex.Class))
myRDFSgraph.add((ex.PortugueseLocation, rdf.type, ex.Class))
myRDFSgraph.add((ex.EuropeanLocation, rdf.type, ex.Class))
myRDFSgraph.add((ex.Food, rdf.type, ex.Class))
myRDFSgraph.add((ex.TraditionalObject, rdf.type, ex.Class))
myRDFSgraph.add((ex.Color, rdf.type, ex.Class))
myRDFSgraph.add((ex.Thing, rdf.type, ex.Class))

# Classes Labels

myRDFSgraph.add((ex.Town, rdfs.label, Literal('Vila', lang="pt")))
myRDFSgraph.add((ex.District, rdfs.label, Literal('Distrito', lang="pt")))
myRDFSgraph.add((ex.Region, rdfs.label, Literal('Região', lang="pt")))
myRDFSgraph.add((ex.PortugueseLocation, rdfs.label, Literal('Localização Portuguesa', lang="pt")))
myRDFSgraph.add((ex.EuropeanLocation, rdfs.label, Literal('Localização Europeia', lang="pt")))
myRDFSgraph.add((ex.Food, rdfs.label, Literal('Comida', lang="pt")))
myRDFSgraph.add((ex.TraditionalObject, rdfs.label, Literal('Objeto Tradicional', lang="pt")))
myRDFSgraph.add((ex.Color, rdfs.label, Literal('Cor', lang="pt")))
myRDFSgraph.add((ex.Thing, rdfs.label, Literal('Coisa', lang="pt")))


# Definition of the subclasses

myRDFSgraph.add((ex.Town, rdfs.subClassOf, ex.PortugueseLocation))
myRDFSgraph.add((ex.District, rdfs.subClassOf, ex.PortugueseLocation))
myRDFSgraph.add((ex.Region, rdfs.subClassOf, ex.PortugueseLocation))
myRDFSgraph.add((ex.PortugueseLocation, rdfs.subClassOf, ex.EuropeanLocation))
myRDFSgraph.add((ex.Food, rdfs.subClassOf, ex.Thing))
myRDFSgraph.add((ex.TraditionalObject, rdfs.subClassOf, ex.Thing))
myRDFSgraph.add((ex.Color, rdfs.subClassOf, ex.Thing))


# Properties

myRDFSgraph.add((ex.isPartOf, rdf.type, rdf.Property))
myRDFSgraph.add((ex.isPartOf, rdfs.domain, ex.District))
myRDFSgraph.add((ex.isPartOf, rdfs.range, ex.Region))

myRDFSgraph.add((ex.hasFood, rdf.type, rdf.Property))
myRDFSgraph.add((ex.hasFood, rdfs.domain, ex.Town))
myRDFSgraph.add((ex.hasFood, rdfs.range, ex.Food))

myRDFSgraph.add((ex.belongsTo, rdf.type, rdf.Property))
myRDFSgraph.add((ex.belongsTo, rdfs.domain, ex.Town))
myRDFSgraph.add((ex.belongsTo, rdfs.range, ex.District))

myRDFSgraph.add((ex.hasTraditionalObject, rdf.type, rdf.Property))
myRDFSgraph.add((ex.hasTraditionalObject, rdfs.domain, ex.Town))
myRDFSgraph.add((ex.hasTraditionalObject, rdfs.range, ex.TraditionalObject))

myRDFSgraph.add((ex.hasTraditionalColor, rdf.type, rdf.Property))
myRDFSgraph.add((ex.hasTraditionalColor, rdfs.domain, ex.Food))
myRDFSgraph.add((ex.hasTraditionalColor, rdfs.range, ex.Color))

myRDFSgraph.add((ex.has, rdf.type, rdf.Property))
myRDFSgraph.add((ex.has, rdfs.domain, ex.Thing))
myRDFSgraph.add((ex.has, rdfs.range, ex.Thing))

# Properties Labels

myRDFSgraph.add((ex.isPartOf, rdfs.label, Literal('Faz Parte De', lang="pt")))
myRDFSgraph.add((ex.hasFood, rdfs.label, Literal('Tem Comida Tradicional', lang="pt")))
myRDFSgraph.add((ex.belongsTo, rdfs.label, Literal('Pertence A', lang="pt")))
myRDFSgraph.add((ex.hasTraditionalObject, rdfs.label, Literal('Tem Objeto Tradicional', lang="pt")))
myRDFSgraph.add((ex.hasTraditionalColor, rdfs.label, Literal('Tem Cor Tradicional', lang="pt")))
myRDFSgraph.add((ex.has, rdfs.label, Literal('Tem', lang="pt")))


# Definition of the subproperties

myRDFSgraph.add((ex.hasTraditionalColor, rdfs.subPropertyOf, ex.has))
myRDFSgraph.add((ex.has, rdfs.subPropertyOf, ex.traditionallyHas))


# Instances

myRDFSgraph.add((ex.Sintra, rdf.type, ex.Town))
myRDFSgraph.add((ex.Barcelos, rdf.type, ex.Town))
myRDFSgraph.add((ex.Lisbon, rdf.type, ex.District))
myRDFSgraph.add((ex.Braga, rdf.type, ex.District))
myRDFSgraph.add((ex.NorthRegion, rdf.type, ex.Region))
myRDFSgraph.add((ex.CenterRegion, rdf.type, ex.Region))
myRDFSgraph.add((ex.SintraPillows, rdf.type, ex.Food))
myRDFSgraph.add((ex.BarcelosRooster, rdf.type, ex.TraditionalObject))
myRDFSgraph.add((ex.Yellow, rdf.type, ex.Color))


# Instances Labels

myRDFSgraph.add((ex.Sintra, rdfs.label, Literal('Vila de Sintra', lang="pt")))
myRDFSgraph.add((ex.Barcelos, rdfs.label, Literal('Município de Barcelos', lang="pt")))
myRDFSgraph.add((ex.Lisbon, rdfs.label, Literal('Lisboa', lang="pt")))
myRDFSgraph.add((ex.Braga, rdfs.label, Literal('Braga', lang="pt")))
myRDFSgraph.add((ex.NorthRegion, rdfs.label, Literal('Região Norte', lang="pt")))
myRDFSgraph.add((ex.CenterRegion, rdfs.label, Literal('Região Centro', lang="pt")))
myRDFSgraph.add((ex.SintraPillows, rdfs.label, Literal('Travesseiros de Sintra', lang="pt")))
myRDFSgraph.add((ex.BarcelosRooster, rdfs.label, Literal('Galo de Barcelos', lang="pt")))
myRDFSgraph.add((ex.Yellow, rdfs.label, Literal('Amarelo', lang="pt")))


# Using properties to relate the instances

myRDFSgraph.add((ex.Sintra, ex.belongsTo, ex.Lisbon))
myRDFSgraph.add((ex.Barcelos, ex.belongsTo, ex.Braga))
myRDFSgraph.add((ex.Lisbon, ex.isPartOf, ex.CenterRegion))
myRDFSgraph.add((ex.Braga, ex.isPartOf, ex.NorthRegion))
myRDFSgraph.add((ex.Sintra, ex.hasFood, ex.SintraPillows))
myRDFSgraph.add((ex.Barcelos, ex.hasTraditionalObject, ex.BarcelosRooster))
myRDFSgraph.add((ex.SintraPillows, ex.hasTraditionalColor, ex.Yellow))


# Comments on Portuguese Locations

myRDFSgraph.add((ex.Sintra, rdfs.comment, Literal('Sintra (Santa Maria e São Miguel, São Martinho e São Pedro de Penaferrim) (oficialmente, União das Freguesias de Sintra (Santa Maria e São Miguel, São Martinho e São Pedro de Penaferrim)) é uma freguesia portuguesa do município de Sintra, com 63,55 km² de área e 29 591 habitantes (2011).', lang='pt')))
myRDFSgraph.add((ex.Barcelos, rdfs.comment, Literal('Barcelos é uma cidade portuguesa da sub-região do Cávado, pertencendo à região Norte e ao distrito de Braga, com 24 177 habitantes (2021) no seu perímetro urbano.', lang='pt')))
myRDFSgraph.add((ex.Lisbon, rdfs.comment, Literal('Lisboa é a capital de Portugal, situada na costa. Do imponente Castelo de São Jorge, a vista abrange as construções em tons pastel da cidade antiga, o estuário do Tejo e a Ponte 25 de Abril.', lang='pt')))
myRDFSgraph.add((ex.Braga, rdfs.comment, Literal('Braga é uma cidade no extremo norte de Portugal, a nordeste do Porto. É conhecida pela herança e eventos religiosos.', lang='pt')))
myRDFSgraph.add((ex.CenterRegion, rdfs.comment, Literal('A Região do Centro ou Região Centro ou Região das Beiras é uma região situada no centro de Portugal, tendo 2.227.912 habitantes em 2021.', lang='pt')))
myRDFSgraph.add((ex.NorthRegion, rdfs.comment, Literal('A Região do Norte ou Região Norte é uma região situada no norte de Portugal, tendo a cidade do Porto como cidade administrativa da região e uma extensão perto de 21 000 km quadrados.', lang='pt')))


print("Now let's check what we can infer from your knowledge graph...")
print("The more rules you cover, the better!")
myRDFSreasoner(myRDFSgraph)

# Saving the graph

save_graph(myRDFSgraph, 'myRDFSgraph.ttl')

Now let's check what we can infer from your knowledge graph...
The more rules you cover, the better!
(rdfs 3)  http://example.com/kad/BarcelosRooster rdf:type http://example.com/kad/TraditionalObject
(rdfs 9)  http://example.com/kad/Sintra rdf:type http://example.com/kad/PortugueseLocation
(rdfs 9)  http://example.com/kad/Barcelos rdf:type http://example.com/kad/PortugueseLocation
(rdfs 11)  http://example.com/kad/Town rdfs:subClassOf http://example.com/kad/EuropeanLocation
(rdfs 9)  http://example.com/kad/Yellow rdf:type http://example.com/kad/Thing
(rdfs 3)  http://example.com/kad/Yellow rdf:type http://example.com/kad/Color
(rdfs 9)  http://example.com/kad/SintraPillows rdf:type http://example.com/kad/Thing
(rdfs 9)  http://example.com/kad/BarcelosRooster rdf:type http://example.com/kad/Thing
(rdfs 5)  http://example.com/kad/hasTraditionalColor rdfs:subPropertyOf http://example.com/kad/traditionallyHas
(rdfs 7)  http://example.com/kad/SintraPillows http://example.com/kad/has http://

## Task 4 (2 points) Compare local inferences with GraphDB results

Upload *myRDFSgraph.ttl* to GraphDB (check [the GraphDB tutorial](https://github.com/ucds-vu/knowledge-data-vu/blob/master/Tutorials/Preliminaries/tutorial-GraphDB.md) before starting to work with GraphDB).

Formulate two different SPARQL queries, and write a Python code that executes these queries over your GraphDB SPARQL endpoint (check example of Task 1).

**Each SPARQL query should return a _different type_ of inferred knowledge** (at least one triple that was not explicitly asserted in the graph).

Specify below next to your query (using a comment '# ...') which type of RDFS rule is the GraphDB reasoner using to infer this answer (rdfs2, rdfs3, rdfs5, rdfs7, rdfs9, rdfs11). 

In [27]:
# Get your GraphDB repository URL (setup -> repositories -> repository url) and assign it to the variable 'myEndpoint' below. 
# It should be similar to this: 

myEndpoint = "http://127.0.0.1:7200/repositories/KnowledgeAndData"  # KnowledgeAndData is the name of the repository
sparql = SPARQLWrapper(myEndpoint)

In [31]:
# Query 1 - Specify which RDFS rule are you testing: 
# Rule rdfs2: If G contains the triples (aaa rdfs:domain xxx.) and (uuu aaa yyy.)  then infer the triple (uuu rdf:type xxx.)

sparql.setQuery("""
    SELECT ?uuu ?xxx
    WHERE {?aaa rdfs:domain ?xxx .
           ?uuu ?aaa ?yyy .
        }    
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for inferred_triples in results["results"]["bindings"]:
    print(f"({inferred_triples['uuu']['value']} rdf:type {inferred_triples['xxx']['value']} .)")    

(http://www.w3.org/1999/02/22-rdf-syntax-ns#Property rdf:type http://www.w3.org/2000/01/rdf-schema#Class .)
(http://www.w3.org/2000/01/rdf-schema#Class rdf:type http://www.w3.org/2000/01/rdf-schema#Class .)
(http://www.w3.org/2000/01/rdf-schema#Literal rdf:type http://www.w3.org/2000/01/rdf-schema#Class .)
(http://www.w3.org/2001/XMLSchema#nonNegativeInteger rdf:type http://www.w3.org/2000/01/rdf-schema#Class .)
(http://www.w3.org/2001/XMLSchema#nonNegativeInteger rdf:type http://www.w3.org/2000/01/rdf-schema#Class .)
(http://www.w3.org/2000/01/rdf-schema#Datatype rdf:type http://www.w3.org/2000/01/rdf-schema#Class .)
(http://www.w3.org/2000/01/rdf-schema#Datatype rdf:type http://www.w3.org/2000/01/rdf-schema#Class .)
(http://www.w3.org/2000/01/rdf-schema#ContainerMembershipProperty rdf:type http://www.w3.org/2000/01/rdf-schema#Class .)
(http://www.w3.org/2000/01/rdf-schema#ContainerMembershipProperty rdf:type http://www.w3.org/2000/01/rdf-schema#Class .)
(http://www.w3.org/1999/02/22-

In [32]:
# Query 2 - Specify which RDFS rule are you testing:
# Rule rdfs5: If G contains the triples (uuu rdfs:subPropertyOf vvv.) and (vvv rdfs:subPropertyOf xxx.) then infer the triple
# (uuu rdfs:subPropertyOf xxx.)

sparql.setQuery("""
    SELECT ?uuu ?xxx
    WHERE {?uuu rdfs:subPropertyOf ?vvv .
           ?vvv rdfs:subPropertyOf ?xxx .
        }    
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for inferred_triples in results["results"]["bindings"]:
    print(f"({inferred_triples['uuu']['value']} rdfs:subPropertyOf {inferred_triples['xxx']['value']} .)")

(http://www.w3.org/1999/02/22-rdf-syntax-ns#type rdfs:subPropertyOf http://www.w3.org/1999/02/22-rdf-syntax-ns#type .)
(http://www.w3.org/2000/01/rdf-schema#subPropertyOf rdfs:subPropertyOf http://www.w3.org/2000/01/rdf-schema#subPropertyOf .)
(http://www.w3.org/2000/01/rdf-schema#subClassOf rdfs:subPropertyOf http://www.w3.org/2000/01/rdf-schema#subClassOf .)
(http://www.w3.org/2000/01/rdf-schema#domain rdfs:subPropertyOf http://www.w3.org/2000/01/rdf-schema#domain .)
(http://www.w3.org/2000/01/rdf-schema#range rdfs:subPropertyOf http://www.w3.org/2000/01/rdf-schema#range .)
(http://www.w3.org/2002/07/owl#equivalentProperty rdfs:subPropertyOf http://www.w3.org/2000/01/rdf-schema#subPropertyOf .)
(http://www.w3.org/2002/07/owl#equivalentClass rdfs:subPropertyOf http://www.w3.org/2000/01/rdf-schema#subClassOf .)
(http://proton.semanticweb.org/protonsys#transitiveOver rdfs:subPropertyOf http://proton.semanticweb.org/protonsys#transitiveOver .)
(http://www.w3.org/2002/07/owl#inverseOf rdf